# feature engineering
Start:
* one database with two collections that hold all the sprints and issues with all the raw data
* only addition to this raw data is a field which repository they belong to

End:

* Sprints:
  * Make aggregation of each sprint's metadata:
    * sprint planned duration in days
    * sprint overdue boolean (completedDate > endDate)
    * Number of issues inside sprint
    * Median number of issues with changed sprint
* Issues:
  * predictor: has sprint field in change log (boolean)
  * Number of sprint fields in change log (int)
  * Add originSprintId field (uuid)
  * Add above sprint metadata fields to every issue
  * All not used (custom)fields should be removed

In [23]:
# Auxiliary
import pymongo
import re
import json
import numpy as np
from datetime import datetime
from dateutil import parser

client = pymongo.MongoClient("mongodb://localhost:27017")
db = client['JiraRepos']

In [25]:
# Sprints
coll = db['02_Sprints']
sprints = coll.find()

# Iterate over each sprint
for sprint in sprints:
    # print(sprint)
    
    # sprint planned duration in days
    try:
        startDate = parser.parse(sprint['startDate'])
        endDate = parser.parse(sprint['endDate'])
        plannedDuration =  (endDate - startDate).days
    except:
        plannedDuration = np.nan

    # sprint overdue boolean (completedDate > endDate)
    try:
        completeDate = parser.parse(sprint['completeDate'])
        if (completeDate - endDate).days > 1:
            isOverdue = 1
        else:
            isOverdue = 0
    except:
        isOverdue = np.nan


    # Number of issues inside sprint
    noOfIssues = len(sprint['issues'])

    print(sprint['id'], plannedDuration, isOverdue, noOfIssues)



Jira2660 0 1 3
Jira2743 10 nan 3
Jira4728 12 0 17
Jira4762 12 1 26
Jira4698 12 0 3
Jira3693 13 0 7
Jira3551 13 0 5
Jira3443 28 1 38
Jira2823 27 0 11
Jira2983 13 0 11
Jira3660 13 0 11
Jira3743 13 0 6
Jira3719 13 0 14
Jira3770 13 1 3
Jira1342 13 1 5
Jira1305 13 0 3
Jira1246 10 1 1
Jira1252 10 1 1
Jira1200 13 1 3
Jira1167 13 0 3
Jira1168 13 0 1
Jira1127 13 1 3
Jira1201 13 1 2
Jira1083 12 0 3
Jira989 13 1 3
Jira901 13 1 1
Jira956 13 1 2
Jira824 13 0 7
Jira823 13 1 3
Jira750 13 0 5
Jira693 13 1 4
Jira692 13 0 8
Jira657 14 1 1
Jira598 20 0 30
Jira630 14 0 3
Jira573 14 0 10
Jira1128 13 1 1
Jira539 15 1 17
Jira515 10 1 34
Jira513 20 1 3
Jira489 14 0 39
Jira496 14 1 2
Jira476 3 1 11
Jira446 10 1 33
Jira459 14 1 2
Jira424 14 0 12
Jira408 13 0 35
Jira389 14 0 32
Jira3635 13 0 2
Jira369 13 0 30
Jira355 13 0 43
Jira336 9 1 32
Jira306 13 0 35
Jira270 14 1 50
Jira28 14 0 31
Jira250 14 0 45
Jira228 14 0 23
Jira203 14 0 48
Jira187 14 0 25
Jira172 14 0 47
Jira148 14 0 32
Jira129 9 1 39
Jira110 14 0 11
J

In [ ]:
# Issues
coll = db['01_Issues']
issues = coll.find()

# Number of sprint fields in change log (int)
# predictor: has sprint field in change log (boolean)
# Add originSprintId field (uuid)
# Add above sprint metadata fields to every issue
# All not used (custom)fields should be removed

In [ ]:
# Sprints that need the new Issue data
# Median number of issues with changed sprint